In [34]:
import sys # delete after finish
sys.path.append('..') # delete after finish

from seismicpro import SeismicDataset, SeismicIndex, Survey, Gather
from seismicpro.batchflow import BA, B
import matplotlib.pyplot as plt

In [26]:
sgy_path = './data/small_survey.sgy'
header_cols = ["TraceNumber", "offset", "INLINE_3D", "CROSSLINE_3D", 'CDP'] # порезать до необходимого минимума
survey = Survey(sgy_path, header_index='FieldRecord', header_cols=header_cols, name='raw')

In [27]:
dataset = SeismicDataset(surveys=survey)

In [49]:
ix = SeismicIndex(surveys=[survey, survey], mode='c')

In [50]:
ix.surveys_dict

{'raw': [<seismicpro.src.survey.Survey at 0x7f96ec994160>,
  <seismicpro.src.survey.Survey at 0x7f96ec994160>]}

In [46]:
dataset.index.headers

INLINE_3D  CROSSLINE_3D      CDP  TraceNumber  offset  \
CONCAT_ID FieldRecord                                                          
0         111916            1607           538  2130094         1656     127   
          111916            1547           517  2050513          195    3304   
          111916            1595           474  2114118         1304    3303   
          111916            1601           604  2122204         1578    3301   
          111916            1601           473  2122073         1447    3301   
...                          ...           ...      ...          ...     ...   
          111920            1565           563  2074427          661    2310   
          111920            1571           576  2082396          818    2308   
          111920            1571           525  2082345          767    2308   
          111920            1577           518  2090294          904    2297   
          111920            1583           500  2098232         1030    2851   

                       TRACE_SEQUENCE_FILE  
CONCAT_ID FieldRecord                       
0         111916                      5185  
          111916                      6344  
          111916                      6343  
          111916                      6342  
          111916                      6341  
...                                    ...  
          111920                      4027  
          111920                      4026  
          111920                      4025  
          111920                      4023  
          111920                      4320  

[8640 rows x 6 columns]

In [45]:
dataset.index.surveys_dict

{'raw': [<seismicpro.src.survey.Survey at 0x7f96ec994160>]}

In [42]:
pipeline = (dataset.p
    .load(src='raw')
    .print(B('raw'))
    .print(BA('raw')['offset'])
#     .load(src='raw2')
#     .call(lambda b : b * 0, B('raw2'), save_to=BA('raw2').data)
#     .update(BA('raw').data, BA('raw2').data)
)

In [43]:
batch = pipeline.next_batch(3)

[[ 127 3304 3303 ... 2324 2327 2314]
 [3335 3328 3328 ... 2355 2380 2345]
 [3266 3274 3274 ... 2275 2295 4914]]


In [37]:
batch.raw

array([<seismicpro.src.gather.Gather object at 0x7f96ec7c2828>],
      dtype=object)

### Gather Tutorial

The Tutorial contains general information about the Gather class, helpful attributes and some useful methods. <br>

### Content

[Intro](#Intro) <br>
[Starting](#Starting) <br>
[Attributes](#Attributes) <br>
[General Methods](#General-Methods)<br>
[Normalization Methods](#Normalization-Methods) <br>
[Conclusion](#Conclusion) <br>

In [ ]:
import sys # delete after finish
sys.path.append('..') # delete after finish

from seismicpro import Survey
import matplotlib.pyplot as plt

### Intro

Пару слов о Survey <br>
Что такое Gather? <br>
В чем отличие от Survey? <br>
Зачем понадобился отдельный объект? <br>
Как помогает Gather? <br>

### Starting
We will consider the creation of a Gather class and tell about it structure.

Gather class based on Survey class [tutorial here]. поднять в Intro <br> 
We need to create Survey first.

In [ ]:
sgy_path = './data/small_survey.sgy'
header_cols = ["TraceNumber", "offset", "INLINE_3D", "CROSSLINE_3D", 'CDP'] # порезать до необходимого минимума
survey = Survey(sgy_path, header_index='FieldRecord', header_cols=header_cols)

#### samle_gather

Now we could create Gather class by using 

In [ ]:
gather = survey.sample_gather()
gather.plot() #show here

Let's see information about the created class object

In [ ]:
gather.info()

Tell about viewed information. Point on `Index name` and `Index value`

(!) tips <br>
Gather statistics auto calcs when `info()` called 

#### get_gather

получаем список уникальных индексов

In [ ]:
survey.headers.index.drop_duplicates()

Объясняем метод `get_gather`

In [ ]:
gather = survey.get_gather(index=111918, limits=(0,3000)) # limits remove maybe...
gather.info()

<b> Notes </b>
если в Сюрвее есть лимиты, то они же будут в Gather

Now we learn how to create `Gather`

### Attributes

Let discuss about most useful attributes of Survey class

<b>`headers`</b> - хэддеры берем из Survey для каждой каждой трасы, попавшей в Gather. <br>

In [ ]:
gather.headers

<b>`data`</b> - trace data, numpy array with shape [Number of traces, Trace length]

In [ ]:
gather.data

In [ ]:
gather.data.shape

<b>`samples`</b> - array of samples with appropriate intervals, ms

In [ ]:
gather.samples

<b>`sample_rate` </b> - time resolution of traces, ms

In [ ]:
gather.sample_rate

We learned about the main attributes of Gather class

`sort_by` и `mask` 
прописывать в методах 

mask using in picking tasks only

### General Methods

General methods

#### <i>plot </i>

show gather information

In [ ]:
#gather = survey.sample_gather().sort(by='offset') # descent?
gather.plot()

#### <i>sort </i>

sort the traces in gather by headers

In [ ]:
gather = survey.sample_gather().sort(by='CDP')
gather.plot()

добавляет атрибут `sort_by`

In [ ]:
gather.sort_by

#### <i>copy </i>

make copy of the gather

Нужно ли сравнение копии и оригинала?
Полезный метод, т.к. все методы работают inplace

In [ ]:
gather_copy = gather.copy()
gather.sort(by='offset')
gather.plot()

In [ ]:
gather_copy.plot()

#### <i>dump </i>

save gather as file in SEG-Y format

In [ ]:
gather.dump(path='./data/', name='test_gather') 

Tell about `name` and `copy_header` params

### Normalization Methods

возможно переименовать. Например Data Processing

#### <i>scale_standard </i>

Приведение значений трасс к среднему 0 и дисперсии 1

All methods works inplace. I use copy to show you how methods work on the base `Gather`.

In [ ]:
gather_scaled = gather.copy()

gather_scaled.scale_standard()
print(f'Mean of all trace is {gather_scaled.data.mean():.2f}')
print(f'Standart deviation of all trace is {gather_scaled.data.std():.2f}')

In [ ]:
_ = plt.hist(gather_scaled.data.ravel(), bins=20)

In [ ]:
gather_scaled = gather.copy()
gather_scaled.scale_standard(use_global=True)
_ = plt.hist(gather_scaled.data.ravel(), bins=20)

In [ ]:
survey.collect_stats()

#### <i>scale_maxabs </i>gather.scale_standard()
gather.info()

In [ ]:
gather_scaled = gather.copy()

gather_scaled.scale_maxabs()
print(f'Min of all trace is {gather_scaled.data.min():.2f}')
print(f'Max of all trace is {gather_scaled.data.max():.2f}')

In [ ]:
_ = plt.hist(gather_scaled.data.ravel(), bins=20)

In [ ]:
gather_scaled = gather.copy()
gather_scaled.scale_maxabs(q_min=0.01, q_max=0.99)
_ = plt.hist(gather_scaled.data.ravel(), bins=20)

In [ ]:
gather_scaled = gather.copy()
gather_scaled.scale_maxabs(q_min=0.01, q_max=0.99, clip=True)
_ = plt.hist(gather_scaled.data.ravel(), bins=20)

#### <i>scale_minmax </i>

In [ ]:
gather_scaled = gather.copy()

gather_scaled.scale_minmax()
_ = plt.hist(gather_scaled.data.ravel(), bins=20)

### Muter

subblock with `mute` and `create_muter` methods

First create `Muter` object with `create_muter` method

only `from_point` mode explain

In [ ]:
gather_muted = gather.copy()
offsets = [50, 1500, 2000, 4000]
times = [500, 2000, 1000, 6000]
points = {'offsets': offsets, 'times': times}
muter = gather_muted.create_muter('points', **points)

Second mute еру traces in `Gather` with `Muter` object

In [ ]:
gather_muted.mute(muter, fill_value=0)

In [ ]:
gather_muted.plot()

#### <i>qet_quantile </i>

return quantile
убрать в низ блока нормализации
упомянуть не более

In [ ]:
gather.get_quantile(0.01)

could return requested quantile of each trace in Gather if `tracewise=True`

In [ ]:
gather.get_quantile(0.01, tracewise=True)

could use Survey statistic if its already calculated with `Survey.collect_stats()` <br>
`gather.get_quantile(0.01, use_global=True)`

In [ ]:
survey.collect_stats()
gather.get_quantile(0.01, use_global=True)

### Conclusion

In this tutorial we looked at `Gather` class <br>

Now we could create `Gather` from `Survey` <br>
Learned about the main attributes of this class <br>
Showed how the main methods works <br>

More information and full documentation you could find [here] <br>

Hope it was been useful for you <br>